In [71]:
#import all relevant packages
import torch, torchvision
import torchvision.transforms as transforms
import numpy as np
from backpack import backpack, extend
from backpack.extensions import DiagHessian
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from scipy.stats import pearsonr
from mpl_toolkits.axes_grid1 import AxesGrid
import mpl_toolkits as mpl
from scipy.stats import chi2, norm
import random
#import warnings
#warnings.filterwarnings("ignore")

plt.rcParams["mpl_toolkits.legacy_colorbar"] = False

s=2
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#plt stuff
plt.rcParams["figure.figsize"] = (20,10)

In [83]:
def NN():
    
    features = torch.nn.Sequential(
        torch.nn.Linear(1, 1),
        #torch.nn.Linear(1, 2)
    )
    return(features)

net = NN()
loss_function = torch.nn.MSELoss()

train_optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=5e-4)

In [119]:
class NumbersDataset(Dataset):
    def __init__(self):
        self.nums = torch.tensor(np.linspace(-1, 1, 500))#torch.rand(-1, 1, 500)
        self.samples = []
        #self.samples = self.nums.clone()
        #self.samples = torch.tensor([self.nums.clone(), self.nums.clone()]).view(2, -1)
        for x in self.nums:
            self.samples.append((x, x))
        #    if num > 0:
        #        self.samples.append((num, 1))
        #    else:
        #        self.samples.append((num, 0))
        

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        #return self.samples[idx]
        num, label = self.samples[idx]
        return num, label
        #$return self.samples[idx]
if __name__ == '__main__':
    dataset = NumbersDataset()
    #dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2)
    #print(len(dataset))
    #print(dataset[99])
    #print(dataset[0])
    for i, batch in enumerate(dataloader):
        print(i, batch)
        if i > 10:
            break
        
trainset, testset = random_split(dataset, [400, 100])

dataloader = DataLoader(trainset, batch_size=5, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=5, shuffle=False, num_workers=2)

0 [tensor([-0.7715,  0.9238,  0.0902, -0.7796, -0.2184], dtype=torch.float64), tensor([-0.7715,  0.9238,  0.0902, -0.7796, -0.2184], dtype=torch.float64)]
1 [tensor([-0.3387, -0.9038,  0.3186, -0.3948,  0.4629], dtype=torch.float64), tensor([-0.3387, -0.9038,  0.3186, -0.3948,  0.4629], dtype=torch.float64)]
2 [tensor([ 0.9760,  0.7435,  0.7234, -0.8437, -0.6433], dtype=torch.float64), tensor([ 0.9760,  0.7435,  0.7234, -0.8437, -0.6433], dtype=torch.float64)]
3 [tensor([ 0.5752,  0.6393, -0.2064, -0.6994,  0.0621], dtype=torch.float64), tensor([ 0.5752,  0.6393, -0.2064, -0.6994,  0.0621], dtype=torch.float64)]
4 [tensor([-0.0461,  0.1824, -0.9198, -0.0902,  0.5792], dtype=torch.float64), tensor([-0.0461,  0.1824, -0.9198, -0.0902,  0.5792], dtype=torch.float64)]
5 [tensor([ 0.7074,  0.5551, -0.1944, -0.5551,  0.2705], dtype=torch.float64), tensor([ 0.7074,  0.5551, -0.1944, -0.5551,  0.2705], dtype=torch.float64)]
6 [tensor([ 0.7635, -0.7074,  0.8838, -0.6353,  0.9078], dtype=torch.f

In [120]:
def get_accuracy(output, targets):
    """Helper function to print the accuracy"""
    predictions = output.argmax(dim=1, keepdim=True).view_as(targets)
    return predictions.eq(targets).float().mean().item()

In [121]:
net = net.float()
def train(verbose=False, num_iter=30):
    max_len = len(dataloader)
    for iter in range(num_iter):
        for batch_idx, (x, y) in enumerate(dataloader):
            y = y.view(-1, 1)
            y = y.float()
            #print(y)
            x = x.view(-1, 1)
            #print(x)
            output = net(x.float())
            #print(output)
            #output_flat = output.view(-1)
            #y_flat = y.view(-1)

            accuracy = get_accuracy(output, y)

            loss = loss_function(output, y)
            loss.backward()
            train_optimizer.step()
            train_optimizer.zero_grad()

            if verbose:
                if batch_idx % 10 == 0:
                    print(
                        "Iteration {}; {}/{} \t".format(iter, batch_idx, max_len) +
                        "Minibatch Loss %.3f  " % (loss) +
                        "Accuracy %.0f" % (accuracy * 100) + "%"
                    )

    #print("saving model at: {}".format(MNIST_PATH))
    #torch.save(mnist_model.state_dict(), MNIST_PATH)

In [122]:
train(verbose=True)

Iteration 0; 0/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 10/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 20/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 30/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 40/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 50/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 60/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 0; 70/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 0/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 10/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 20/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 30/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 40/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 50/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 60/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 1; 70/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 2; 0/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 2; 10/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 2; 20/80 	Minibatch L

Iteration 19; 0/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 10/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 20/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 30/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 40/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 50/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 60/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 19; 70/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 0/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 10/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 20/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 30/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 40/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 50/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 60/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 20; 70/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 21; 0/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 21; 10/80 	Minibatch Loss 0.000  Accuracy 0%
Iteration 21;

In [123]:
for (x, y) in testloader:
            y = y.view(-1, 1)
            y = y.float()
            #print(y)
            x = x.view(-1, 1)
            #print(x)
            output = net(x.float())
            #print(output)
            #output_flat = output.view(-1)
            #y_flat = y.view(-1)

            accuracy = get_accuracy(output, y)
            print(accuracy)
            print(output)
            print(x)

0.0
tensor([[-0.1182],
        [-0.7830],
        [-0.0861],
        [-0.6989],
        [-0.9312]], grad_fn=<AddmmBackward>)
tensor([[-0.1182],
        [-0.7836],
        [-0.0862],
        [-0.6994],
        [-0.9319]], dtype=torch.float64)
0.0
tensor([[ 0.9472],
        [-0.7670],
        [-0.1742],
        [-0.3745],
        [ 0.9352]], grad_fn=<AddmmBackward>)
tensor([[ 0.9479],
        [-0.7675],
        [-0.1743],
        [-0.3747],
        [ 0.9359]], dtype=torch.float64)
0.0
tensor([[ 0.7830],
        [ 0.6388],
        [-0.5547],
        [-0.5467],
        [-0.1983]], grad_fn=<AddmmBackward>)
tensor([[ 0.7836],
        [ 0.6393],
        [-0.5551],
        [-0.5471],
        [-0.1984]], dtype=torch.float64)
0.0
tensor([[ 0.6788],
        [-0.9632],
        [-0.5347],
        [-0.2263],
        [ 0.3544]], grad_fn=<AddmmBackward>)
tensor([[ 0.6794],
        [-0.9639],
        [-0.5351],
        [-0.2265],
        [ 0.3547]], dtype=torch.float64)
0.0
tensor([[ 0.4586],
        [

In [124]:
state = net.state_dict()
print(state)

OrderedDict([('0.weight', tensor([[0.9993]])), ('0.bias', tensor([-8.1095e-06]))])
